In [1]:
import sys
sys.path.append('..')
import numpy as np
import math
from qiskit.circuit import QuantumCircuit,QuantumRegister
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector
# from AdaptVQE_.My_adapt import QubitAdapt,MyAdaptVQE
from AdaptVQE_.MyQubit_Adapt_VQE import QubitAdapt
from AdaptVQE_.My_adapt import MyAdaptVQE
from AdaptVQE_.QEBMapper.QEBMapper import QEBMapper,QEB_Operator
# Define Molecule
molecule = MoleculeInfo(
    ["H", "H"], [(0.0, 0.0, 0.0), (0.8*1, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
#problem = transformer.transform(problem=problem)

In [2]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD,UCC

uccsd = UCCSD(qubit_converter=QubitConverter(JordanWignerMapper()),
              num_particles=problem.num_particles,
              num_spatial_orbitals=problem.num_spatial_orbitals,
              initial_state=HartreeFock(num_particles=problem.num_particles,num_spatial_orbitals=problem.num_spatial_orbitals,qubit_converter=converter),
              generalized=False  # 如果设为True ==> UCCGSD G=generalized
              )

#print(f'H2的SD激发列表是:{uccsd.excitation_list},\nH2的激发费米子算符是:\n{uccsd.excitation_ops()}\n{uccsd.decompose().draw()}')

In [4]:

#Qubit产生算符
def Q_dagger(index:int,length:int):
    if index==0:
        operator=(X-1j*Y)
    else:
        operator=I
    for i in range(1,length):
        if index==i:
            operator = (X-1j*Y)^operator
        else:
            operator= I^operator
    return operator
    #Qubit-湮灭算符
    #index从0开始 高比特位在前 低位在后：eg: a2 = IXII +1j IYII
def Q(index:int,length:int):
    if index==0:
        operator=(X+1j*Y)
    else:
        operator=I
    for i in range(1,length):
        if index==i:
            operator = (X+1j*Y)^operator
        else:
            operator= I^operator
    return operator

In [ ]:
a1 = Q(index=1,length=4)
a1_dagger = Q_dagger(index=1,length=4)
a0 = Q(index=0,length=4)
a0_dagger = Q_dagger(index=0,length=4)
###验证反对易性质
a1

In [3]:
QEBMapper(Pauli_length=4,Excitationlist=uccsd.excitation_list)

1.0 * IIXX
+ -1j * IIYX
+ 1j * IIXY
+ 1.0 * IIYY
- 1.0 * IIXX
+ -1j * IIYX
+ 1j * IIXY
- 1.0 * IIYY
1.0 * XIII
+ -1j * YIII
- 1.0 * XIII
+ -1j * YIII
1.0 * XIII
+ -1j * YIII
- 1.0 * XIII
+ -1j * YIII


In [25]:
for i in uccsd.excitation_list:
    print(f'i={i}\n')
    for index,k in enumerate(i):
        print(f'i[index]={i[index][0]},index={index},k={k[0]}')
        

i=((0,), (1,))

i[index]=0,index=0,k=0
i[index]=1,index=1,k=1
i=((2,), (3,))

i[index]=2,index=0,k=2
i[index]=3,index=1,k=3
i=((0, 2), (1, 3))

i[index]=0,index=0,k=0
i[index]=1,index=1,k=1


In [31]:
i=((0,), (1,))
i[0][0]

0

In [8]:
uccsd.excitation_list[0]

((0,), (1,))

In [54]:
from qiskit.opflow import I, X, Y
def Q_dagger(index: int,length:int):
    if index==0:
        operator=(X-1j*Y)
    else:  
        operator=I            
    for i in range(1,length):
        print(f'op={operator},i={i}')
        if index == i:
            operator = (X-1j*Y)^operator
        else:
            operator = I^operator
            
    return operator


In [55]:
Q_dagger(index=2,length=4)

op=I,i=1
op=II,i=2
op=1.0 * XII
+ -1j * YII,i=3


PauliSumOp(SparsePauliOp(['IXII', 'IYII'],
              coeffs=[1.+0.j, 0.-1.j]), coeff=1.0)

In [35]:
Q_dagger(index=0,length=4)

PauliSumOp(SparsePauliOp(['XIII', 'YIII'],
              coeffs=[1.+0.j, 0.-1.j]), coeff=1.0)

In [29]:
for i in range(1,4):
    print(i)

1
2
3
